In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

include("../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Hand Visualization

In [5]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.00, -0.01, 0.31701]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.0
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.31701
 0.0  0.0  0.0   1.0

In [42]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "world")
#display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [215]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))

Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [3]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiples springs/dampers

Virtual Mechanism system

In [4]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

"world"

Repulsive cylinder

In [5]:
cylinder_radius = 0.025
cylinder_length = 0.2
cylinder_position = SVector(0.0, -0.045, 0.32)

add_coordinate!(vms,  ConstCoord(cylinder_position);  id="cylinder position")
add_coordinate!(vms, ConstCoord(cylinder_radius); id="cylinder radius")

add_coordinate!(vms, FramePoint(".robot.rh_palm", SVector(0. , 0., 0.07)); id="second palm point")

repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_mftip_mass_coord", ".robot.rh_rftip_mass_coord",".robot.rh_lftip_mass_coord" , 
                    ".robot.rh_thtip_mass_coord", ".robot.rh_ffmiddle_mass_coord",".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord",
                    ".robot.rh_lfmiddle_mass_coord",  ".robot.rh_thmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_mfproximal_mass_coord",
                    ".robot.rh_rfproximal_mass_coord", ".robot.rh_lfproximal_mass_coord", ".robot.rh_thproximal_mass_coord", ".robot.rh_palm_mass_coord", "second palm point")
frames_names = ("fftip", "mftip", "rftip", "lftip", "thtip", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox", "palm", "palm2")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, "cylinder position") ; id = "$(frames_names[i]) cylinder diff" )
    add_coordinate!(vms, CoordSlice("$(frames_names[i]) cylinder diff", SVector(2,3)); id="$(frames_names[i]) planar error")
    add_coordinate!(vms, CoordNorm("$(frames_names[i]) planar error") ; id = "$(frames_names[i]) planar error norm")
    add_coordinate!(vms, CoordDifference("$(frames_names[i]) planar error norm", "cylinder radius"); id = "shifted $(frames_names[i]) cylinder error" )

    add_component!(vms, ReLUSpring(1000.0, "shifted $(frames_names[i]) cylinder error", true); id="$(frames_names[i]) cylinder repulsive spring")
    add_component!(vms, RectifiedDamper(100.0, "$(frames_names[i]) planar error norm", (0.0, 1.1*cylinder_radius), true, false); id="$(frames_names[i]) cylinder damper")
end

Hand motion

In [6]:
# TARGET REACHING
add_coordinate!(vms,  ConstCoord(cylinder_position);  id="Target position")

add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "Target position"); id="ff position error")
add_coordinate!(vms, CoordDifference(".robot.rh_mftip_mass_coord", "Target position"); id="mf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_rftip_mass_coord", "Target position"); id="rf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_lftip_mass_coord", "Target position"); id="lf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_thtip_mass_coord", "Target position"); id="th position error")

K_matrix = SMatrix{3, 3}(10., 0., 0., 0., 10., 0., 0., 0., 10.)
D_matrix = SMatrix{3, 3}(10., 0., 0., 0., 10., 0., 0., 0., 10.)

add_component!(vms, LinearSpring(K_matrix, "ff position error"); id="ff spring")
add_component!(vms, LinearDamper(D_matrix, "ff position error"); id="ff damper")

add_component!(vms, LinearSpring(K_matrix, "mf position error"); id="mf spring")
add_component!(vms, LinearDamper(D_matrix, "mf position error"); id="mf damper")

add_component!(vms, LinearSpring(K_matrix, "rf position error"); id="rf spring")
add_component!(vms, LinearDamper(D_matrix, "rf position error"); id="rf damper")

add_component!(vms, LinearSpring(K_matrix, "lf position error"); id="lf spring")
add_component!(vms, LinearDamper(D_matrix, "lf position error"); id="lf damper")

add_component!(vms, LinearSpring(K_matrix, "th position error"); id="th spring")
add_component!(vms, LinearDamper(D_matrix, "th position error"); id="th damper")

"th damper"

## Simulating the Robot

### Setting Up the Simulation

In [7]:
tspan = (0., 4.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)
@info "Simulating shadow robot with repulsive cylinder"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-5, reltol=1e-5);

[ Info: Simulating shadow robot with repulsive cylinder
ODE   0%|█                                              |  ETA: N/A
ODE  23%|███████████                                    |  ETA: 0:00:33
ODE  80%|██████████████████████████████████████         |  ETA: 0:00:05
ODE 100%|███████████████████████████████████████████████| Time: 0:00:21


### Visualizing the Results

In [9]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))


robotvisualize!(ls, plotting_kcache)

X = SVector(1., 0., 0.)
cylinder = Cylinder(Point3f(cylinder_position + (cylinder_length/2)*X), Point3f(cylinder_position - (cylinder_length/2)*X), cylinder_radius)
mesh!(ls, cylinder; color=:magenta, transparency=true)


savepath = joinpath(module_path, "docs/src/assets/shadowhand_cylinder_collision_model.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);